In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import Row, SQLContext, SparkSession
from pyspark import SparkConf
# Import data types
from pyspark.sql.types import *
conf = SparkConf().set("spark.network.timeout","800s").set("spark.driver.memory","100g").set("spark.memory.storageFraction","0.2")
spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate()

In [ ]:
#multipleline
pcapPath = "/home/antslab/data_hdd4t/中華電信/pcap/snort.2020-01-10_dir/snort.log/outbound_*.json"
pcap_df_multiline = spark.read.option("multiline", "true").json(pcapPath)

In [ ]:
# 重新整理schema
# layer_df = pcap_df_multiline.select("_source.layers.`ip.src`")
# src_ip_df = layer_df.select("`ip.src`")

In [ ]:
src_ip_df = pcap_df_multiline.select("_source.layers.`ip.src`")
src_ip_df.createOrReplaceTempView("src_ip")

In [ ]:
maxflowquery = (spark.sql("SELECT `ip.src`, COUNT(*) FROM src_ip GROUP BY `ip.src` ORDER BY COUNT(*) DESC LIMIT 10"))

In [ ]:
# 直接拿maxflowquery來畫圖
result = maxflowquery.collect()

In [ ]:
import pickle as pk
with open("cht_0110_top10.pk", 'wb') as file:
    pk.dump(result, file)